surprise 모듈을 설치

In [1]:
conda install surprise

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - surprise

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [2]:
!wget "https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/movies.csv"
!wget "https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/ratings.csv"

--2022-09-06 15:38:52--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/movies.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.56.39
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.56.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458390 (448K) [text/csv]
Saving to: 'movies.csv'

movies.csv          100%[===================>] 447.65K  --.-KB/s    in 0.05s   

2022-09-06 15:38:52 (8.11 MB/s) - 'movies.csv' saved [458390/458390]

--2022-09-06 15:38:52--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/movielens/ratings.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.56.39
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.56.39|:443... connected.
HTT

In [3]:
!ls -tl

total 129384
-rw-r--r--  1 KBS  staff     21088  9  6 15:37 Item-based_Recommendation.ipynb
-rw-r--r--  1 KBS  staff     29181  9  6 15:27 Item-based_Collaborative_filtering_Recommendation_Systems.ipynb
-rw-r--r--  1 KBS  staff    123277  9  6 15:16 User-based_Collaborative-filtering_Recommendation.ipynb
-rw-r--r--  1 KBS  staff     18726  9  6 12:32 User-based_Recommendation.ipynb
-rw-r--r--  1 KBS  staff    180208  9  6 10:34 TF-IDF_Contents_Based_Recommendation_System.ipynb
-rw-r--r--  1 KBS  staff     17697  9  6 09:48 TF-IDF_Cosine&Person_Similarity.ipynb
-rw-r--r--@ 1 KBS  staff    100861  9  5 18:41 TMDB_Recommendation.ipynb
-rw-r--r--  1 KBS  staff   5698602  9  5 17:03 tmdb_5000_movies.csv
-rw-r--r--  1 KBS  staff  40044293  9  5 17:03 tmdb_5000_credits.csv
-rw-r--r--  1 KBS  staff    458390  2 23  2021 movies.csv
-rw-r--r--  1 KBS  staff   2438266  2 23  2021 ratings.csv
-rw-r--r--  1 KBS  staff   2438266  2 23  2021 ratings.csv.1
-rw-r--r--  1 KBS  staff   2438266  2 23  202

# 데이터 로딩


In [4]:
from surprise import Dataset
from surprise import Reader

from collections import defaultdict
import numpy as np
import pandas as pd

In [5]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
movie_ratings = pd.merge(movies, ratings, left_on='movieId', right_on='movieId')

# 딥러닝에서 시간순으로 훈련을 시킬 수 있다.
# 시간 별로 정보가 다르기 때문에.

In [9]:
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [10]:
movies.shape

(9125, 3)

In [11]:
ratings.shape

(100004, 4)

In [12]:
movie_ratings.shape

(100004, 6)

In [14]:
def getMovieName(movie_ratings, movieID):
    return movie_ratings[movie_ratings["movieId"] == movieID][["title", "genres"]].values[0]

def getMovieID(movie_ratings, movieName):
    return movie_ratings[movie_ratings["title"] == movieName][["movieId", "genres"]].values[0]

## 영화 데이터를 surprise 모듈을 통해 로딩

In [15]:
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1) # sep은 분리자, 따옴표,콤마면 csv이고, skipline은 헤더무시해라

data = Dataset.load_from_file("ratings.csv", reader=reader) # data에 저장

In [16]:
from surprise import KNNBasic
import heapq
from collections import defaultdict

In [18]:
# 데이터를 훈련용과 테스트용으로 나누지 말고 모두 리턴
trainSet = data.build_full_trainset()

In [19]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}

# 아이템 기반과 코사인 유사도 기반으로 동작하는 모델 생성
model = KNNBasic(sim_options=sim_options)
# 데이터로부터 아이템 벡터 생성
model.fit(trainSet)
# 아이템간의 유사도 행렬 계산
simsMatrix = model.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [21]:
testUserId = '85'
k = 10

In [22]:
pd.set_option('display.max_colwidth', -1)
display(movie_ratings[movie_ratings["userId"] == int(testUserId)].sort_values("rating", ascending=False))

/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,movieId,title,genres,userId,rating,timestamp
263,2,Jumanji (1995),Adventure|Children|Fantasy,85,5.0,837511784
7443,255,"Jerky Boys, The (1995)",Comedy,85,5.0,837512635
19363,648,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller,85,5.0,837511448
17405,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,85,5.0,837507044
14386,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,85,5.0,837511743
...,...,...,...,...,...,...
16429,546,Super Mario Bros. (1993),Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,85,1.0,837512482
11769,366,"Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994)",Drama|Horror|Mystery|Thriller,85,1.0,837512580
12156,374,Richie Rich (1994),Children|Comedy,85,1.0,837512723
8420,291,Poison Ivy II (1996),Drama|Thriller,85,1.0,837512611


In [23]:
testUserInnerID = trainSet.to_inner_uid(testUserId)

In [24]:
# 이 사용자가 좋아한 영화 k개를 읽어서 kNeighbors에 저장
testUserRatings = trainSet.ur[testUserInnerID]
kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

In [25]:
# 이 사용자 좋아한 영화들과 비슷한 영화를 찾아서 candidates에 유사도 가중치를 곱해서 저장
candidates = defaultdict(float)
for itemID, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating)        

In [26]:
# 사용자가 이미 본 아이템들을 기록
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

In [27]:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=lambda t: t[1], reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(movieID, getMovieName(movie_ratings, int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break

3136 ['James Dean Story, The (1957)' 'Documentary'] 50.0
2607 ['Get Real (1998)' 'Drama|Romance'] 49.93620560356324
259 ['Kiss of Death (1995)' 'Crime|Drama|Thriller'] 49.83440938875971
998 ['Set It Off (1996)' 'Action|Crime'] 49.818661078285594
1935 ['How Green Was My Valley (1941)' 'Drama|Musical|Romance'] 49.71992040532635
1440 ['Amos & Andrew (1993)' 'Comedy'] 49.69868472501265
269 ['My Crazy Life (Mi vida loca) (1993)' 'Drama'] 49.691452437730206
988 ['Grace of My Heart (1996)' 'Comedy|Drama'] 49.631279483226095
2068 ['Fanny and Alexander (Fanny och Alexander) (1982)'
 'Drama|Fantasy|Mystery'] 49.628498357279526
896 ['Wild Reeds (Les roseaux sauvages) (1994)' 'Drama'] 49.58113202209386
2626 ['Edge of Seventeen (1998)' 'Comedy|Drama|Romance'] 49.565143823458385


In [30]:
def recommendForItem(userID):
    testUserInnerID = trainSet.to_inner_uid(userID)
    testUserRatings = trainSet.ur[testUserInnerID]

    kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

    candidates = defaultdict(float)
    for itemID, rating in kNeighbors:
        similarityRow = simsMatrix[itemID]
        for innerID, score in enumerate(similarityRow):
            candidates[innerID] += score * (rating) 
 
    watched = {}
    for itemID, rating in trainSet.ur[testUserInnerID]:
        watched[itemID] = 1

    pos = 0
    for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not itemID in watched:
            movieID = trainSet.to_raw_iid(itemID)
            print(movieID, getMovieName(movie_ratings, int(movieID)), ratingSum)
            pos += 1
            if (pos > 10):
                break      

In [31]:
recommendForItem('85')

NameError: name 'itemgetter' is not defined